# The Functools Module

The `functools` module provides functions that act on other functions. It's a bit of a random assortment of utilities that do very different things, but they do it in a sort of similar way.

What's covered here is what I personally thought was useful or interesting. There is more to the `functools` module, which you can read about in the Python docs [here](https://docs.python.org/3/library/functools.html). Here's what *is* in this presentation: 

1. Background
2. `reduce`
3. `partial`
4. `cache`
5. `lru_cache`
6. `singledispatch`

Much of what's left is identical to what's here, except the decorators apply to instance methods, not functions (more on that later). The last bits of functionality that aren't covered by those parallels have to do with updating docstrings for wrapped functions on the fly (`@wraps`) and an esoteric way to create key functions (`@cmp_to_key`). 

## A little background review

In Python, functions are "first class objects", meaning that you can pass them around in code just like other data types.

In this example, we'll create a function whose job it is to call another function. 



In [8]:
import random

def function_caller(other_function):
    """ Accepts a function, announces it will run the function, then run it """
    print(f"About to run the function '{other_function.__name__}'!\n")
    result = other_function()
    print(f"Ran '{other_function.__name__}' and the result was: {result}")

def get_random_number():
    return random.randint(0, 100)

# function_caller(get_random_number)

# remember that when you pass functions around,
# you should pass a reference to the function, not call it!
function_caller(get_random_number) 



AttributeError: 'int' object has no attribute '__name__'

Notice that we pass a *reference* to `get_random_number`, not the result of it. If we did that, the function would fail. In this case, it fails with an `AttributeError` on the very first line, since an `int` does not have a `__name__` attribute.

One other concept that's going to be helpful in learning `functools` is decorators. Decorators are a nice shorthand way to write **functions that return other functions**. Let's say you want some function to only run on a certain day of the week, but you can't or don't want to check the time manually everywhere it's called. You could wrap each call in another function that checks whether it can run:

In [10]:
import time

def only_run_on_thursday(other_function):
    """ Checks whether it's Thursday, and if so, calls its argument"""
    if time.localtime().tm_wday == 3: # range [0, 6], Monday is 0
        other_function()
    else: 
        print(f"Could not run {other_function.__name__}, it's not Thursday!")
    
def say_hello():
    print("Hello there!")

only_run_on_thursday(say_hello)

Could not run say_hello, it's not Thursday!


That's a little clunky, so Python introduced decorators to eliminate some of that wordiness. We can get exactly the same functionality as above with the following code:

In [11]:
import time

def only_run_on_thursday(other_function):
    def wrapper():
        if time.localtime().tm_wday == 3: # range [0, 6], Monday is 0
            other_function()
        else: 
            print(f"Could not run {other_function.__name__}, it's not Thursday!")
    return wrapper
    
@only_run_on_thursday
def say_hello():
    print("Hello there!")

say_hello()

Hello there!


This is nice, because callers of `say_hello` don't have to worry about whether it's Thursday or not. They can just call the function, and the decorator will take care of doing the time check every time. 

There's a lot of technical detail in exactly *how* this happens under the surface, but that's beyond the scope of this presentation. 

## Common Use Cases

### functools.reduce

When is passing functions around useful? Sometimes when you want to invoke a function many times on a set of arguments!

For example, you might want to `reduce` a set of numbers to a single value. A common example for teaching this is to get the product of a list of numbers, but we'll do something a little more interesting.

Let's reduce a list using an algorithm that adds even numbers and subtracts odd numbers:

In [15]:
import functools

def add_evens_subtract_odds(current_value, next_value):
    print(f"Current: {current_value} \t Next: {next_value}")
    if next_value % 2 == 0:
        return current_value + next_value
    return current_value - next_value

numbers = [6, 5, 2, 1]

# result = functools.reduce(add_evens_subtract_odds, numbers)

# reduce also accepts a starting value:
# result = functools.reduce(add_evens_subtract_odds, numbers, 10)

# it's a good idea to use a default, because passing an empty list
# to `reduce` results in an error:
functools.reduce(add_evens_subtract_odds, [], 0)


0

It's a little weird that Python doesn't provide this as a built-in. Such is life! 

# functools.partial

Sometimes you want to temporarily redefine a function with specific arguments. This can be handy if you want to use a generic function in a specific way. Calling `functools.partial` will return a new function with some of the arguments pre-defined.

Here's an academic example:

In [18]:
def add_two_numbers(a, b, third=0):
    return a + b

add_ten = functools.partial(add_two_numbers, 10, third=7)
add_ten(1) # should print 11

11

The `partial` function returns a `partial` object. If you want, you can inspect it to see what's going on under the surface:

In [20]:
# print(add_ten)
print("Original function:\t", add_ten.func)
print("Fixed arguments:\t", add_ten.args)
print("Fixed keywords:\t\t", add_ten.keywords)

# note that these are attributes added by `partial`. If you look for them on the original function, they don't exist:
# print(add_two_numbers.func) # Attribute error!


AttributeError: 'function' object has no attribute 'func'

Here's a more practial example, direct from the Python docs. It creates a function that parses a string into a base 2 number instead of the default base 10:

In [21]:
basetwo = functools.partial(int, base=2)

print(int("101010")) # defaults to base 10
int("101010", base=2)

def my_function():
    pass

basetwo("10100")

101010


20

## Caching values

`functools` offers a few methods to cache values. There are 5, but we'll just look at these two:

- The `@cache` decorator, which stores the return value of a function for a given set of arguments. This is useful for speeding up deeply recursive functions. 
- The `@lru_cache` decorator, which is the same as `cache`, but it puts a limit on how big the cache can grow. If the cache is about to get too big, it drops the **L**east **R**ecently **U**sed value and adds the new one.

Here's an example of how you might use `@cache`:

(Note: remember that decorators are just functions that return other functions!)

In [24]:
# Example from the docs:
@functools.cache
def factorial(n):
    return n * factorial(n-1) if n else 1

factorial(10)      # no previously cached result, makes 11 recursive calls
factorial(5)       # just looks up cached value result
factorial(12)      # makes two new recursive calls, the other 10 are cached


# But let's prove it! We'll add a counter that gets incremented every time our function is called.
@functools.cache
def monitored_factorial(n):
    global num_calls
    # print("Called monitored_factorial with argument", n)
    num_calls += 1
    return n * monitored_factorial(n-1) if n else 1

num_calls = 0
monitored_factorial(10)
print(num_calls) # Should print 11


num_calls = 0
monitored_factorial(5)
print(num_calls) # Should print 0

num_calls = 0
monitored_factorial(12)
print(num_calls) # Should print 2


11
0
2


{'maxsize': None, 'typed': False}

`@lru_cache` is similar, but it restricts the total number of cached items that can be stored. `@cache` is actually the equivalent of using `lru_cache` with `maxsize=None`. 

LRU caching is a good optimization to make when the results of the cached function are likely to favor a few commonly-requested values. For example, if your program reports the weather based on a location, you might want to cache the results for the user's local zip code, as well as a few nearby areas. 


In [28]:
# NOTE: to use this cell, you have to install theh `requests` library
# and the 'toml' library (for pip, that's `pip install requests`)

# You will also have to create a free account with tomorrow.io and get an api key

import json
import functools
import requests
import pprint
import toml 

config = toml.load("secrets.toml")
BASE_URL = "https://api.tomorrow.io/v4/timelines"
API_KEY = config["API_KEY"]

@functools.lru_cache(maxsize=2, typed=True)
def request_weather(latitude, longitude):
    querystring = {
    "location": f"{str(latitude)}, {str(longitude)}", 
    "fields":["temperature"],
    "units":"metric",
    "timesteps":"1d",
    "apikey":API_KEY}

    response = requests.request("GET", BASE_URL, params=querystring)
    return json.loads(response.text)

request_weather(42, 71) # Boston
print(request_weather.cache_info())

request_weather(90, 135) # North Pole
print(request_weather.cache_info())

request_weather(0, 0) # Null Island
print(request_weather.cache_info())

request_weather(42, 71) # Boston again
print(request_weather.cache_info())


CacheInfo(hits=0, misses=1, maxsize=2, currsize=1)
CacheInfo(hits=0, misses=2, maxsize=2, currsize=2)
CacheInfo(hits=0, misses=3, maxsize=2, currsize=2)
CacheInfo(hits=0, misses=4, maxsize=2, currsize=2)


If you need to, you can also clear the cache completely:

In [ ]:
request_weather.cache_clear() # empty cache to start

request_weather(42, 71) # Boston
request_weather(0, 0)
print(request_weather.cache_info()) # should print two misses and no hits
request_weather(42, 71) # Boston 
print(request_weather.cache_info()) # should print one hit and the two misses from before

request_weather.cache_clear()

request_weather(42, 71) # Boston 
print(request_weather.cache_info()) # we expect to see one miss because the cache is empty again.

`lru_cache` caches values based on function inputs, and by default it ignores type:

In [ ]:
my_cache: {
    (42, 71): "cloudy",
    (0, 0): "very cold",
    (0.0, 0): "super cold"
}

request_weather.cache_clear()

request_weather(0, 0)
print(request_weather.cache_info())
request_weather(0.0, 0)
request_weather(0.0, 0.0)

# TODO: update after request limit expires


## Single Dispatch functions

Some languages support *function overloading* natively, meaning you can define a function using the same name but different arguments, and at compile time, the correct implementation will be chosen based on which arguments are used. Here's an example in C++:

```
// Source: https://www.geeksforgeeks.org/function-overloading-c/
#include <iostream>
using namespace std;
 
// this function definition accepts 2 ints
void add(int a, int b)
{
  cout << "sum = " << (a + b);
}
 
// this one accepts two doubles
void add(double a, double b)
{
    cout << endl << "sum = " << (a + b);
}
 
int main()
{
    // here, we don't have to specify which 'add' to use, it "just works"
    add(10, 2);
    add(5.3, 6.2);
 
    return 0;
}
```

Python lets you call a function with differently-typed arguments, but it's a kind of blind acceptance. The function won't behave any differently if you pass in arguments with incorrect types.

In [30]:
def add(a, b):
    return a + b

# We can call this function with any arguments we'd like!
print(add(1, 2)) # expect 3
print(add(1.0, 2.0)) # expect 3.0
print(add("1", "2")) #...whoops, 12

3
3.0
12


This is where `singledispatch` comes in. In this framework, we can register a parent function to define the name/interface we want, then register multiple other variants to handle different argument types. At runtime, the interpeter will pick the appropriate function based on which arguments are passed in. 

This is function overloading! (with a few extra steps)

In [34]:
@functools.singledispatch
def generic_add(a, b):
    return a + b

# called when string args are provided
@generic_add.register
def _(a:str, b:str):
    print("strings")
    return int(a) + int(b)

# called when list args are provided
@generic_add.register(list)
def _(a, b):
    return sum(a + b)

print(generic_add(1, 2))
print(generic_add("1", "2"))
print(generic_add([1, 2], [3, 4]))


3
strings
3
10


One important thing to note is that the interpreter will only look at the first argument to the generic function to choose the implementation to run. (this is no matter which version of type hinting you use. I was wrong about this in the presentation!)

In [36]:
    
generic_add("1", 3) # this will choose the function that was registered for strings

strings


4

But if we define the arguments with explicit type hints:

In [ ]:
@generic_add.register
def _(a: list, b: int):
    return sum(a) + b

generic_add([1], 2)

## That's all, folks!

That's the end of the notebook, at least. Take a look at the `exercises` folder in [this repo](https://github.com/awordforthat/functools_demo) for practice!